In [1]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Concatenate, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import add
from keras import regularizers
import keras

import pandas as pd

import cv2 as cv

import numpy as np

Using TensorFlow backend.


# Model Definition

## Encoder
The role of this model is to create feature vector from raw images.

Borrow from https://github.com/uzh-rpg/rpg_public_dronet

Note:
    Layer naming convention: `layer_name(str) + stage(int) + block(str)`

### Define a convolutional block

In [2]:
def convolutional_block(X, num_filters, shape_filters, strides, stage):
    """
    Implementation of convolutional block in Residual network
    
    Input:
        X (tensor): input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        num_filters (list of 3 ints): list of number of filters
        shape_filters (list of 3 ints): list of filters' shape
        strides (list of 3 ints): list of strides
        stage (int): stage of this convolutional block in the whole ResNet
        
    Output:
        tensor of shape (m, n_H, n_W, n_C)
    """
    
    # retrieve filters shape from filters
    n1, n2, n3 = num_filters
    f1, f2, f3 = shape_filters
    
    # retrieve strides from strides
    s1, s2, s3 = strides
    
    # create name
    bn_name_base = 'bn_' + str(stage) + '_'
    conv_name_base = 'conv_' + str(stage) + '_'
    
    # save value of X
    X_shorcut = X
    
    # First component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'a')(X)
    X = Activation('relu')(X)
    X = Conv2D(n1, (f1, f1), strides=[s1, s1], padding='same',
               name=conv_name_base + 'a',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Second component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'b')(X)
    X = Activation('relu')(X)
    X = Conv2D(n2, (f2, f2), strides=[s2, s2], padding='same',
               name=conv_name_base + 'b',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Short-cut
    X_shorcut = Conv2D(n3, (f3, f3), strides=[s3, s3], padding='same', name=conv_name_base + 'c')(X_shorcut)
    
    X = add([X, X_shorcut])
    
    return X


### Assemble 3 convolutional block to make a ResNet 

In [3]:
def resnet8_clean(input_shape):
    """
    Define encoder architecture as ResNet8
    
    Input:
        input_shape (list of ints): shape of input image [n_H, n_W, n_C]
        
    Output:
        model: a Model instance
    """
    
    # Input
    X_input = Input(shape=input_shape)
    
    # Apply 1st convolution & max pooling on input
    X = Conv2D(32, (5, 5), strides=[2,2], padding='same', name='conv_0')(X_input)
    X = MaxPooling2D(pool_size=(3, 3), strides=[2,2])(X) 
    
    # First convolutional block
    X = convolutional_block(X, [32, 32, 32], [3, 3, 1], [2, 1, 2], stage=1)
    
    # Second convolutional block
    X = convolutional_block(X, [64, 64, 64], [3, 3, 1], [2, 1, 2], stage=2)
    
    # Third convolutional block
    X = convolutional_block(X, [128, 128, 128], [3, 3, 1], [2, 1, 2], stage=3)
    
    # Output layer
    X = Flatten()(X)
    X = Activation('relu')(X)
    
    # Define model
    model = Model(inputs=[X_input], outputs=[X])
    print(model.summary())
    
    return model

### Create model & load weights

In [4]:
encoder = resnet8_clean([200, 200, 1])
encoder.load_weights('./model/named_resnet8_best_weights.h5', by_name=True)

# freeze weights of encoder
for l in encoder.layers:
    l.trainable = False
    
encoder.compile(optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

In [5]:
# This is needed to add encoder to computation graph
_X = np.random.randn(1, 200, 200, 1)
encoder.predict(_X)

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Decoder

This model decode the feature vector created by the Encoder to produce the prediction for the `PLANNING_HORIZON` (in form of classes of steering angles) 

In [13]:
def fully_connected(input_shape, planning_horizon):
    """
    Define a network of 3 dense layer
    
    Input:
        input_shape (list of ints)
        num_classes (int)
        
    Output:
        Model instance
    """
    X_input = Input(shape=input_shape)
    
    X = Dropout(0.5)(X_input)
    
    X = Dense(500, activation='relu', kernel_regularizer=regularizers.l2(1e-2))(X)
    
    X = Dropout(0.5)(X)
    
    X = Dense(200, activation='relu', kernel_regularizer=regularizers.l2(1e-2))(X)
     
    y = Dense(planning_horizon, activation=None)(X)
        
    model = Model(inputs=[X_input], outputs=[y])
    print(model.summary())
    
    return model
    

In [14]:
decoder = fully_connected([3 * 6272], 25)

decoder.compile(loss='mse', optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 18816)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 18816)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               9408500   
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_6 (Dense)              (None, 25)                5025      
Total params: 9,513,725
Trainable params: 9,513,725
Non-trainable params: 0
_________________________________________________________________


# Data Generator

This class preprocesses training examples, form traning batches and fit batches to network 

In [8]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dataset_csv, batch_size=1, 
                 encoder_input_shape=(200, 200), 
                 encoder_output_shape=6272, 
                 shuffle=True):
        """
        Input:
            dataset_csv (string): path to csv contains dataset
            batch_size (int): size of a training batch
            encoder_input_shape (tuple): shape of tensor inputted to encoder
            encoder_output_shape (int): size of vector outputted by encdoer
            shuffle (bool): shuffle dataset after 1 epoch
        """
        
        self.df = pd.read_csv(dataset_csv)
        self.batch_size = batch_size
        self.img_shape = encoder_input_shape
        self.encoder_output_shape = encoder_output_shape
        self.shuffle = shuffle
        # invoke on_epoch_end to create shuffle training dataset
        self.on_epoch_end()
    
    def __len__(self):
        """
        Output:
            the number of batches per epoch
        """
        return int(np.floor(len(self.df) / self.batch_size))
    
    def __data_generation(self, list_indexes):
        """
        Input:
            list_indexes (list): list of indexes of training sample in this batch
        
        Output:
            X (np.ndarray): feature vector provided by encoder, shape (batch_size, 3*6272)
            y (np.ndarray): label vector, shape (batch_size, 25)
        """
        
        X = np.zeros((self.batch_size, 3*6272))
        y = np.zeros((self.batch_size, 25))
        
        img_path_prefix = '/home/user/Bureau/Dataset/udacity/'
        
        # Iterate through each idx in training batch
        for i, idx in enumerate(list_indexes): 
            file_names_list = self.df.iloc[idx].X[2: -2].split("', '")
            # generate feature vector for each training sample in the batch
            encoder_X = np.zeros((3, self.img_shape[0], self.img_shape[1], 1))
            for j, name in enumerate(file_names_list):
                # read image
                img = cv.imread(img_path_prefix + name, 0)
                
                # resize & reshape image
                img = np.float32(cv.resize(img, self.img_shape))
                if len(img.shape) == 2:
                    img = img.reshape((img.shape[0], img.shape[1], 1))
                
                # add img to input tensor of encoder
                encoder_X[j, :, :, :] = img
            
            # pass tensor of imgs through encoder to get features vector 
            X[i, :] = encoder.predict(encoder_X).reshape(1, -1)
            
            # get label
            y[i, :] = np.array([float(angle) for angle in self.df.iloc[idx].steering_angles[1: -1].split(", ")])
        return X, y
    
    def __getitem__(self, index):
        """
        Generate one batch of data
        
        Input:
            index (int): index of the first training sample
        """
        # Generate indexes of the batch
        list_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        # if (index + 1) * self.batch_size > len(self.indexes), 
        # list_indexes = [index * self.batch_size: len(self.indexes)]

        # Generate data
        X, y = self.__data_generation(list_indexes)

        return X, y
    
    def on_epoch_end(self):
        """
        Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.df))  # array of indexes of training dataset
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
                    

# Training

In [16]:
# parameters of datagenerator
params = {'batch_size': 5, 
         'encoder_input_shape': (200, 200), 
         'encoder_output_shape': 6272, 
         'shuffle': True}

training_generator = DataGenerator('./data/hybrid_training.csv', **params)
validation_generator = DataGenerator('./data/hybrid_validation.csv', **params)

decoder.fit_generator(generator=training_generator,
                      validation_data=validation_generator,
                      epochs=10,
                      verbose=1)

Epoch 1/10
687/687 [==============================] - 74s 108ms/step - loss: 4.1370 - val_loss: 2.0704
Epoch 2/10
687/687 [==============================] - 73s 106ms/step - loss: 1.6676 - val_loss: 1.2827
Epoch 3/10
687/687 [==============================] - 73s 106ms/step - loss: 1.0162 - val_loss: 0.7609
Epoch 4/10
687/687 [==============================] - 74s 108ms/step - loss: 0.5837 - val_loss: 0.4254
Epoch 5/10
687/687 [==============================] - 73s 106ms/step - loss: 0.3385 - val_loss: 0.2424
Epoch 6/10
687/687 [==============================] - 73s 106ms/step - loss: 0.1867 - val_loss: 0.1343
Epoch 7/10
687/687 [==============================] - 73s 106ms/step - loss: 0.1099 - val_loss: 0.0814
Epoch 8/10
687/687 [==============================] - 73s 106ms/step - loss: 0.0747 - val_loss: 0.0582
Epoch 9/10
687/687 [==============================] - 73s 106ms/step - loss: 0.0559 - val_loss: 0.0445
Epoch 10/10
687/687 [==============================] - 73s 106ms/step - l

In [17]:
# save decoder weights
decoder.save_weights('./model/decoder_weights_MAY11.h5')